#### Задача 1 (2 балла). 

Многие программисты python оформляют свои скрипты таким образом, чтобы можно было запускать их прямо из командной строки, не открывая интерпретатор питона, как python myscript.py. Но в такие скрипты часто бывает нужно передавать дополнительные переменные, например, чтобы запустить наш скрипт для конвертации из семинарской задачи 2, нужно передать путь к файлу, а возможно, имя для сконвертированного  txt. 

Для таких штук используется библиотека argparse: она встроенная (ее не нужно устанавливать), она умеет распарсивать аргументы, полученные из командной строки. То есть, например, я могу прямо в командной строке/терминале написать:

    python myconverter.py myfile.doc newfile.txt

И тогда скрипт myconverter.py (обратите внимание: это работает только с py-скриптами) запустится сам, возьмет файл doc и превратит его в txt с указанным именем. 

Квест: попробуйте самостоятельно разобраться с библиотекой argparse и переписать задачу 2 из семинара в py-скрипте таким образом, чтобы она запускалась из командной строки. Py-скрипт можно прислать мне на почту. 

In [ ]:
# эту задачу надо выполнять в py. С расширением py работает, например, Visual Studio Code или PyCharm Community, в колабе эту задачу сделать не получится. 

#### Задача 2 (3 балла).

На самом деле для формата conllu существует целых две библиотеки python, которые умеют его распарсивать. Поищите эти библиотеки и освойте любую из них. Дополните вашу задачку из второго домашнего задания таким образом, чтобы внутри вашего класса теперь использовалась любая из этих библиотек для чтения (тогда, как можно заметить, класс Token не понадобится - в этих библиотеках он уже реализован). Обычно эти библиотеки имеют метод типа parse, который возвращает объект специального класса (это итератор: если по нему итерироваться, он выдает по очереди все предложения, а внутри предложения можно точно так же проитерироваться по токенам). Получается, в вашем классе теперь должен лежать этот объект из библиотеки, а в методах вы только итерируетесь по нему и считаете все, что вам нужно. 

In [ ]:
import conllu

class Sentence:
    def __init__(self, token_list):
        self.token_list = token_list
        self.text = token_list.metadata.get('text', '')

class Reader:
    def __init__(self):
        self.sentences = []

    def read(self, file_path):
        with open(file_path, 'r', encoding='utf8') as file:
            data = file.read()
            self.sentences = list(conllu.parse(data))

    def upos_statistics(self):
        all_upos = {}  # словарь с подсчетом частей речи
        total = 0  # общее кол-во
        for sentence in self.sentences:
            for token in sentence:
                upos = token['upostag']
                if upos != '_':
                    all_upos[upos] = all_upos.get(upos, 0) + 1
                    total += 1
        print('Статистика частей речи')
        for upos, amount in all_upos.items():
            percentage = (amount / total) * 100
            print(f'{upos:<{max(len(x) for x in all_upos.keys())}}: {round(percentage, 2)}%')

    def find_by_lemma(self, lemma):
        res = []
        for sentence in self.sentences:
            for token in sentence:
                if token['lemma'] == lemma:
                    data = [token['form'], sentence.metadata.get('text', '')]
                    res.append(data)
        return res
    
    def print_sentences(self):
        for i, sentence in enumerate(self.sentences):
            print(f"{i+1}:", end="") #можно было сделать в одну строчку, но у меня мозг расплавился к этому моменту
            print(sentence.metadata.get('text', ''))

    def print_sentence_by_id(self, sentence_id):
        if sentence_id > 0 and sentence_id <= len(self.sentences):
            sentence = self.sentences[sentence_id - 1]
            print(f"Предложение {sentence_id}:")
            print(sentence.metadata.get('text', ''))
            print("Токены:")
            for token in sentence:
                print(f"{token['id']}: {token['form']} ({token['lemma']}, {token['upostag']})")
        else:
            print("Номер предложения вне диапазона.")

reader = Reader()

while True:
    print('Выберите, что вы хотите сделать\n\t0 - выйти\n\t1 - прочитать файл\n\t2 - вывести статистику по частям речи\n\t3 - искать вхождения по лемме\n\t4 - вывести предложения\n\t5 - вывести предложение с токенами по номеру')
    user = int(input())
    if user == 0:  # exit
        break
    elif user == 1:  # reading
        user_path = '../02-OOP/fr_fqb-ud-test.conllu'
        # user_path = input('Введите путь в файлу')
        reader.read(user_path)
    elif user == 2:  # statistics
        reader.upos_statistics()
    elif user == 3:  # search by lemma
        user_lemma = input('Введите лемму')
        result = reader.find_by_lemma(user_lemma)
        print(f'Количество вхождений: {len(result)}')
        for token in result:
            print(f'Найдено: {token[0]} в предложении {token[1]}')
    elif user == 4:  # print sentences
        if reader.sentences:
            reader.print_sentences()
        else:
            print("Сначала прочитайте файл.")
    elif user == 5:  # print sentence by id
        if reader.sentences:
            sentence_id = int(input("Введите номер предложения: "))
            reader.print_sentence_by_id(sentence_id)
        else:
            print("Сначала прочитайте файл.")
    else:
        print('Такой команды нет.')

Выберите, что вы хотите сделать
	0 - выйти
	1 - прочитать файл
	2 - вывести статистику по частям речи
	3 - искать вхождения по лемме
	4 - вывести предложения
	5 - вывести предложение с токенами по номеру
Выберите, что вы хотите сделать
	0 - выйти
	1 - прочитать файл
	2 - вывести статистику по частям речи
	3 - искать вхождения по лемме
	4 - вывести предложения
	5 - вывести предложение с токенами по номеру
Предложение 149:
Quel est le titre du premier volume de la biographie d'Elvis Presley ?
Токены:
1: Quel (quel, ADJ)
2: est (être, AUX)
3: le (le, DET)
4: titre (titre, NOUN)
(5, '-', 6): du (_, _)
5: de (de, ADP)
6: le (le, DET)
7: premier (premier, ADJ)
8: volume (volume, NOUN)
9: de (de, ADP)
10: la (le, DET)
11: biographie (biographie, NOUN)
12: d' (de, ADP)
13: Elvis (Elvis, PROPN)
14: Presley (Presley, PROPN)
15: ? (?, PUNCT)
Выберите, что вы хотите сделать
	0 - выйти
	1 - прочитать файл
	2 - вывести статистику по частям речи
	3 - искать вхождения по лемме
	4 - вывести предложения